In [16]:
#imports
from imgaug import augmenters as iaa
import imgaug as ia
import os
import numpy as np
import cv2
import glob
from PIL import Image
from imgaug.augmentables.bbs import BoundingBox, BoundingBoxesOnImage


In [4]:
# seed for ia randomizer
ia.seed(1)

In [5]:
# fetch certain file types
images_names = []
ignored = 0

for fname in os.listdir('images'):
    if fname.endswith('.jpg'):
        images_names.append(fname)
    else:
        ignored += 1

print("found", len(images_names), ".jpg files")
print("ignored", ignored, "other file types")

found 9 .jpg files
ignored 9 other file types


In [6]:
# extract only bounding box information from .txt file (see format below)
def extract_bounding_boxes(file):
    bounding_boxes = []
    with open(file) as f:
        boxes = f.readlines()
        for box in boxes:
            box_arr = box.split(' ')            
            bounding_boxes.append((box_arr[1], box_arr[2], box_arr[3], box_arr[4].strip("\n")))
    return bounding_boxes


Bounding box .txt files should look as follows:

3 0.297656 0.615278 0.118750 0.205556   
3 0.622656 0.374306 0.056250 0.093056

Where the first number represents a classification label

In [7]:
# put images in np-array from memory & put corresponding bounding boxes in array in order
# bounding_box_images contains tuples in which the first element corresponds to the image and the second element corresponds to the bounding boxes details as an array
bounding_box_images = []

for image_name in images_names:
    # obtain image
    location = "images/"+image_name
    image = cv2.imread (location)
    
    # obtain bounding boxes
    box_location = os.path.splitext(location)[0] + '.txt'
    bounding_boxes = extract_bounding_boxes(box_location)
    
    # put data in tuple and append to array
    bounding_box_images.append((image, bounding_boxes))



print("images found with bounding box(es):", len(bounding_box_images))



images found with bounding box(es): 9


In [62]:
# sequential operations

seq = iaa.Sequential([
    iaa.Crop(px=(0, 16)), # crop images from each side by 0 to 16px (randomly chosen)
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    #iaa.GaussianBlur(sigma=(0, 3.0)) # blur images with a sigma of 0 to 3.0
])

In [65]:
# Augment BBs and images.

# function takes image with corresponding bounding boxes and performs augmentation on both.
def augment_bb_image(img, bbs):
    return seq(image=img, bounding_boxes=bbs)

# transforms pixel data from bounding boxes to normalized yolo values
def normalize_bounding_boxes(img_shape, bbs):
    width = img_shape[0]
    heigth = img_shape[1]

    normalized_bbs = []
    for bb in bbs:
        bb_normalized = (round(bb[0]/width, 6), round(bb[1]/heigth, 6), round(bb[2]/width, 6), round(bb[3]/heigth, 6))
        normalized_bbs.append(bb_normalized)
    return normalized_bbs

In [71]:
# perform image & bounding box augmentation on all images
augmented_bb_images = []
for bb_img in bounding_box_images:

    # extract bounding boxes per image and create BoundingBox objects from data
    bbs = BoundingBoxesOnImage([
        BoundingBox(x1=bb[0], y1=bb[1], x2=bb[2], y2=bb[3]) for bb in bb_img[1]
    ], shape=bb_img[0].shape)

    # augmented_bb_images

    aug_img, aug_bbs = augment_bb_image(bb_img[0], bbs)

    # normalize bounding boxes to yolo format
    aug_bbs = [(bb.x1, bb.y1, bb.x2, bb.y2) for bb in aug_bbs]
    aug_bbs = normalize_bounding_boxes(bb_img[0].shape, aug_bbs)
    aug_bb_img = (aug_img, aug_bbs)

    augmented_bb_images.append(aug_bb_img)





original bbs: [[[113 147 100]
  [108 142  95]
  [105 141  95]
  ...
  [232 237 238]
  [236 241 240]
  [250 251 249]]

 [[119 151 102]
  [116 148  99]
  [113 147 100]
  ...
  [245 244 246]
  [250 248 247]
  [255 250 247]]

 [[126 156 103]
  [125 155 102]
  [122 153 102]
  ...
  [251 245 246]
  [254 247 244]
  [255 246 242]]

 ...

 [[ 53  49  54]
  [ 53  47  52]
  [ 51  44  49]
  ...
  [154 218 159]
  [152 216 157]
  [152 216 157]]

 [[ 53  44  47]
  [ 57  46  49]
  [ 61  47  51]
  ...
  [155 219 160]
  [154 217 161]
  [156 217 161]]

 [[ 59  45  49]
  [ 54  39  43]
  [ 54  37  41]
  ...
  [156 219 163]
  [158 219 163]
  [158 219 163]]]
augmented bbs: [[[109 144  94]
  [106 140  93]
  [108 144  98]
  ...
  [221 234 207]
  [243 249 230]
  [251 249 239]]

 [[ 99 136  86]
  [ 97 133  86]
  [101 138  92]
  ...
  [233 246 222]
  [246 252 235]
  [250 247 242]]

 [[ 98 134  87]
  [ 95 132  86]
  [ 96 135  90]
  ...
  [206 212 193]
  [249 251 239]
  [251 248 244]]

 ...

 [[ 25  25  20]
  [ 58 

In [ ]:
# save augmented images & corresponding bounding boxes in the 'augmented_bb_images' folder

for index,img in enumerate(images_aug):
    try:
        name = "AUGMENTED"+images_names[index]
        cv2.imwrite(os.path.join(path , name), img)
    except:
        print("something went wrong when saving an augmented image to 'augmented_images'")